# Exploration of the Paley-Wiener Thm

In this notebook, we briefly explore numerically the result of a variant of the Paley-Wiener theorem: if $f$ is analytic and $2\pi$-periodic, then there exists $\alpha > 0$ such that $|\hat{f}_k| \leq C e^{-\alpha |k|}$. The $\alpha$ can be chosen such that $f$ is analytic in the strip $\Omega_\alpha := \{ z + i y : |y| \leq \alpha \}$. 

In [ ]:
using Pkg; Pkg.activate(".")
using QuadGK, LinearAlgebra, LaTeXStrings, CairoMakie

In [ ]:
"""
This is a very naive method to compute the 
Fourier coefficients taken from our "naive"
spectral method. We will soon learn about 
much more efficient schemes based on the FFT.
"""
function compute_fcoeffs(f_fun, N)
    fhat = zeros(ComplexF64, 2*N+1)
    kgrid = -N:N
    for (i, k) in enumerate(kgrid)
        g = x -> f_fun(x) * exp(-im * k * x)
        fhat[i] = quadgk(g, -pi, pi; rtol = 1e-6, atol=1e-6)[1] / (2*pi)
    end
    return fhat, kgrid
end

Our model problem is a periodic version of the Witch of Agnesi, 
$$
  f(x) = \frac{1}{1+ c^2 \sin^2(x/2)}
$$
It's analytic extension into the complex plane has a singularity 
at $z = \pm i \alpha$, where $\alpha = \sinh^{-1}(2/c)$.

In [ ]:
f, α = let c = 3
    (x -> 1 / (1 + c^2 * sin(x/2)^2)), asinh(2/c)
end

xp = range(-π, π, length=200)
fig0 = Figure(size = (400, 250)); ax0 = Axis(fig0[1,1])
lines!(xp, f.(xp)); fig0

In [ ]:
F̂, kk = compute_fcoeffs(f, 20);

fig1 = Figure(size = (400, 250))
ax1 = Axis(fig1[1,1], yscale = log10, xlabel = L"|k|", ylabel = L"|\hat{f}_k|")
scatter!(abs.(kk), abs.(F̂), label = L"\hat{f}_k")
lines!( [10, 18], 0.1*exp.(-α * [10, 18]), linestyle = :dash, color = :black, label = L"\exp(-\alpha N)")
axislegend(ax1, position = :rt)
fig1 

We can quickly confirm the resulting rate of approximation.

In [ ]:
function eval_trig(x, uhat) 
    N = (length(uhat)-1) ÷ 2
    return real( sum(uhat[k+N+1] * exp(im * x * k) for k = -N:N) )
end

NN = [4, 8, 12, 16, 20, 24, 28]
xe = range(-π, π, length=1000)
f_ex = f.(xe)
err_max = zeros(length(NN))
for (i, N) in enumerate(NN)
    Ĝ, _ = compute_fcoeffs(f, N);
    fN = eval_trig.(xe, Ref(Ĝ))
    err_max[i] = norm(f_ex - fN, Inf)
end


In [ ]:
fig2 = Figure(size = (350, 250))
ax2 = Axis(fig2[1,1], yscale = log10, xlabel = "N", ylabel = L"|| f - f_N ||_{L^\infty}")
scatterlines!(NN, err_max, label = "error")
lines!(NN[4:end], exp.(-α * NN[4:end]), linestyle = :dash, color = :black, label = L"\exp(-\alpha N)")
axislegend(ax2, position = :rt)
fig2

### Continuously Differentiable Functions

What are the analogous results for (several times) continuously differentiable functions? We can look at a simple example that modifies the Agnesi function a little bit.
$$
f(x) = \frac{1}{1 + 10 |\sin(x/2)|^{5/2}}
$$

In [ ]:
f2 = x -> 1 / (1 + 10 * abs(sin(x/2))^(5/2))

xp = range(-π, π, length=200)
fig3 = Figure(size = (400, 250)); ax3 = Axis(fig3[1,1])
lines!(xp, f2.(xp))
lines!(xp, f.(xp), alpha = 0.5) # previous example
fig3


In [ ]:
F̂2, kk2 = compute_fcoeffs(f2, 40);

fig4 = Figure(size = (400, 250))
ax4 = Axis(fig4[1,1], xscale = log10, yscale = log10, xlabel = L"|k|", ylabel = L"|\hat{f}_k|", limits = (1, 45, nothing, nothing))
scatter!(ax4, abs.(kk2), abs.(F̂2), label = L"\hat{f}_{2,k}")
scatter!(ax4, abs.(kk), abs.(F̂), label = L"\hat{f}_{1,k}")
lines!(ax4, kk2[50:end], 0.7*abs.(kk2[50:end]).^(-7/2), linestyle = :dash, color = :black)
lines!(ax4, kk[30:end], 0.1*exp.(-α * kk[30:end]), linestyle = :dash, color = :black)
axislegend(ax4, position = :rt)
fig4 